<a href="https://colab.research.google.com/github/Sherazkarim1/API-deployment/blob/main/Finance_Chatbot_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install transformers datasets sentencepiece


In [ ]:
import pandas as pd

# Create a simple finance-related dataset
data = {
    "conversation_id": [1, 1, 2, 2, 3, 3],
    "text": [
        "What is compound interest?",
        "Compound interest is the interest calculated on the initial principal and also on the accumulated interest.",
        "How can I calculate my loan EMI?",
        "You can calculate your EMI using the formula: EMI = [P x R x (1+R)^N]/[(1+R)^N-1], where P is the principal, R is the rate, and N is the tenure.",
        "What is a mutual fund?",
        "A mutual fund is a pool of money collected from many investors to invest in securities like stocks, bonds, and other assets."
    ],
    "response": [
        "Compound interest is calculated on both principal and previously earned interest.",
        "It helps your investments grow exponentially over time.",
        "Use an EMI calculator or the formula: EMI = [P x R x (1+R)^N]/[(1+R)^N-1].",
        "Online EMI calculators make this easy!",
        "A mutual fund pools investments to diversify across various securities.",
        "It's managed by professional fund managers."
    ]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Save the dataset as a CSV
df.to_csv("finance_data.csv", index=False)

# Display the dataset
df.head()


,conversation_id,text,response
0,1,What is compound interest?,Compound interest is calculated on both princi...
1,1,Compound interest is the interest calculated o...,It helps your investments grow exponentially o...
2,2,How can I calculate my loan EMI?,Use an EMI calculator or the formula: EMI = [P...
3,2,You can calculate your EMI using the formula: ...,Online EMI calculators make this easy!
4,3,What is a mutual fund?,A mutual fund pools investments to diversify a...


In [ ]:
from datasets import load_dataset

# Load the dataset into Hugging Face's Dataset format
dataset = load_dataset("csv", data_files="finance_data.csv")

# Split into training and validation sets
dataset = dataset["train"].train_test_split(test_size=0.2)
train_dataset = dataset["train"]
val_dataset = dataset["test"]


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
from transformers import AutoTokenizer

# Load the GPT-2 tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Add a padding token to GPT-2 tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=50)

# Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Set the dataset format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",        # Save the model at the end of each epoch
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_steps=500,               # This can remain, but it will only apply if `save_strategy="steps"`
    save_total_limit=2,
    logging_dir="./logs",
    fp16=True,                    # Use mixed precision for faster training
    load_best_model_at_end=True   # Load the best model at the end of training
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Save the model and tokenizer
model.save_pretrained("./finance_chatbot_model")
tokenizer.save_pretrained("./finance_chatbot_model")


('./finance_chatbot_model/tokenizer_config.json',
 './finance_chatbot_model/special_tokens_map.json',
 './finance_chatbot_model/vocab.json',
 './finance_chatbot_model/merges.txt',
 './finance_chatbot_model/added_tokens.json',
 './finance_chatbot_model/tokenizer.json')

In [ ]:
def chat_with_bot():
    print("Chatbot: Hi! I'm MElvi Finance Assistant. Ask me anything about finance.")
    print("Type 'exit' to end the chat.")
    while True:
        # Get user input
        prompt = input("You: ")

        # Exit condition
        if prompt.lower() == 'exit':
            print("Chatbot: Goodbye! Have a great day!")
            break

        # Generate response from the chatbot
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=50)
        inputs['attention_mask'] = inputs['attention_mask']  # Ensure attention_mask is set
        outputs = model.generate(
            inputs.input_ids,
            attention_mask=inputs['attention_mask'],  # Pass attention mask explicitly
            max_length=100,
            num_return_sequences=1,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id  # Explicitly set pad token
        )
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Print chatbot response
        print(f"Chatbot: {response}")


In [ ]:

chat_with_bot()


Chatbot: Hi! I'm MElvi Finance Assistant. Ask me anything about finance.
Type 'exit' to end the chat.
You: What percentage of my income should I save?
Chatbot: What percentage of my income should I save?

The answer is: you should save at least 50% of your income.

If you're a single person, you should save at least 50% of your income.

If you're a married couple, you should save at least 50% of your income.

If you're a married couple, you should save at least 50% of your income.

If you're a single person, you should save at least


KeyboardInterrupt: Interrupted by user